# Affinity Analysis
## Import dataset 

In [1]:
import numpy as np
X = np.loadtxt("affinity_dataset.txt")

In [2]:
n_samples, n_features = X.shape
print("This dataset has {0} samples and {1} features".format(n_samples, n_features))

This dataset has 100 samples and 5 features


In [3]:
# The names of the features, for your reference.
features = ["bread", "milk", "cheese", "apples", "bananas"]

### In our first example, we will compute the Support and Confidence of the rule "If a person buys Apples, they also buy Bananas".

In [4]:
# First, how many rows contain our premise: that a person is buying apples
num_apple_purchases = 0
for sample in X:
    if sample[3] == 1:  # This person bought Apples
        num_apple_purchases += 1
print("{0} people bought Apples".format(num_apple_purchases))

36 people bought Apples


### How many of the cases that a person bought Apples involved the people purchasing Bananas too? Record both cases where the rule is valid and is invalid.

In [5]:
rule_valid = 0
rule_invalid = 0
for sample in X:
    if sample[3] == 1:         # This person bought Apples
        if sample[4] == 1:     # This person bought both Apples and Bananas
            rule_valid += 1
        else:                  # This person bought Apples, but not Bananas
            rule_invalid += 1
            
print("{0} cases of the rule being valid were discovered".format(rule_valid))
print("{0} cases of the rule being invalid were discovered".format(rule_invalid))

21 cases of the rule being valid were discovered
15 cases of the rule being invalid were discovered


### Now we have all the information needed to compute Support and Confidence

In [6]:
support = rule_valid  # The Support is the number of times the rule is discovered.
confidence = rule_valid / num_apple_purchases
print("The support is {0} and the confidence is {1:.3f}.".format(support, confidence))

The support is 21 and the confidence is 0.583.


### Confidence can be thought of as a percentage using the following:


In [7]:
print("As a percentage, that is {0:.1f}%.".format(100 * confidence))

As a percentage, that is 58.3%.


### Now compute for all possible rules

In [8]:
from collections import defaultdict

valid_rules = defaultdict(int)
invalid_rules = defaultdict(int)
num_occurences = defaultdict(int)

In [9]:
for sample in X:
    for premise in range(n_features):
        if sample[premise] == 0: continue
        num_occurences[premise] += 1        # Record that the premise was bought in another transaction
        for conclusion in range(n_features):
            if premise == conclusion:       # It makes little sense to measure if X -> X.
                continue
            if sample[conclusion] == 1:     # This person also bought the conclusion item
                valid_rules[(premise, conclusion)] += 1
            else:                           # This person bought the premise, but not the conclusion 
                invalid_rules[(premise, conclusion)] += 1

In [10]:
print(num_occurences)
print(valid_rules)
print(invalid_rules)

defaultdict(<class 'int'>, {2: 41, 3: 36, 4: 59, 0: 27, 1: 46})
defaultdict(<class 'int'>, {(2, 3): 25, (2, 4): 27, (3, 2): 25, (3, 4): 21, (4, 2): 27, (4, 3): 21, (0, 1): 14, (0, 3): 5, (1, 0): 14, (1, 3): 9, (3, 0): 5, (3, 1): 9, (0, 2): 4, (2, 0): 4, (1, 4): 19, (4, 1): 19, (0, 4): 17, (4, 0): 17, (1, 2): 7, (2, 1): 7})
defaultdict(<class 'int'>, {(2, 0): 37, (2, 1): 34, (3, 0): 31, (3, 1): 27, (4, 0): 42, (4, 1): 40, (0, 2): 23, (0, 4): 10, (1, 2): 39, (1, 4): 27, (3, 2): 11, (3, 4): 15, (0, 1): 13, (2, 4): 14, (1, 0): 32, (1, 3): 37, (4, 2): 32, (4, 3): 38, (0, 3): 22, (2, 3): 16})


### Compute support and confidence

In [11]:
support = valid_rules
confidence = defaultdict(float)
for premise, conclusion in valid_rules.keys():
    confidence[(premise, conclusion)] = valid_rules[(premise, conclusion)] / num_occurences[premise]

### Print all possible rules with corresponding support and confidence

In [12]:
for premise, conclusion in confidence:
    premise_name = features[premise]
    conclusion_name = features[conclusion]
    print("Rule: If a person buys {0} they will also buy {1}".format(premise_name, conclusion_name))
    print(" - Confidence: {0:.3f}".format(confidence[(premise, conclusion)]))
    print(" - Support: {0}".format(support[(premise, conclusion)]))
    print("")

Rule: If a person buys cheese they will also buy apples
 - Confidence: 0.610
 - Support: 25

Rule: If a person buys cheese they will also buy bananas
 - Confidence: 0.659
 - Support: 27

Rule: If a person buys apples they will also buy cheese
 - Confidence: 0.694
 - Support: 25

Rule: If a person buys apples they will also buy bananas
 - Confidence: 0.583
 - Support: 21

Rule: If a person buys bananas they will also buy cheese
 - Confidence: 0.458
 - Support: 27

Rule: If a person buys bananas they will also buy apples
 - Confidence: 0.356
 - Support: 21

Rule: If a person buys bread they will also buy milk
 - Confidence: 0.519
 - Support: 14

Rule: If a person buys bread they will also buy apples
 - Confidence: 0.185
 - Support: 5

Rule: If a person buys milk they will also buy bread
 - Confidence: 0.304
 - Support: 14

Rule: If a person buys milk they will also buy apples
 - Confidence: 0.196
 - Support: 9

Rule: If a person buys apples they will also buy bread
 - Confidence: 0.139
 

### User defined function to print a rule

In [13]:
def print_rule(premise, conclusion, support, confidence, features):
    premise_name = features[premise]
    conclusion_name = features[conclusion]
    print("Rule: If a person buys {0} they will also buy {1}".format(premise_name, conclusion_name))
    print(" - Confidence: {0:.3f}%".format(confidence[(premise, conclusion)]*100))
    print(" - Support: {0}".format(support[(premise, conclusion)]))
    print("")

In [14]:
premise = 1
conclusion = 3
print_rule(premise, conclusion, support, confidence, features)

Rule: If a person buys milk they will also buy apples
 - Confidence: 19.565%
 - Support: 9



### Sorting rules

In [15]:
from pprint import pprint
pprint(list(support.items()))

[((2, 3), 25),
 ((2, 4), 27),
 ((3, 2), 25),
 ((3, 4), 21),
 ((4, 2), 27),
 ((4, 3), 21),
 ((0, 1), 14),
 ((0, 3), 5),
 ((1, 0), 14),
 ((1, 3), 9),
 ((3, 0), 5),
 ((3, 1), 9),
 ((0, 2), 4),
 ((2, 0), 4),
 ((1, 4), 19),
 ((4, 1), 19),
 ((0, 4), 17),
 ((4, 0), 17),
 ((1, 2), 7),
 ((2, 1), 7)]


### Sorting by support 

In [16]:
from operator import itemgetter
sorted_support = sorted(support.items(), key=itemgetter(1), reverse=True)

In [17]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_support[index][0]
    print_rule(premise, conclusion, support, confidence, features)

Rule #1
Rule: If a person buys cheese they will also buy bananas
 - Confidence: 65.854%
 - Support: 27

Rule #2
Rule: If a person buys bananas they will also buy cheese
 - Confidence: 45.763%
 - Support: 27

Rule #3
Rule: If a person buys cheese they will also buy apples
 - Confidence: 60.976%
 - Support: 25

Rule #4
Rule: If a person buys apples they will also buy cheese
 - Confidence: 69.444%
 - Support: 25

Rule #5
Rule: If a person buys apples they will also buy bananas
 - Confidence: 58.333%
 - Support: 21



### Sorting by confidence

In [18]:
sorted_confidence = sorted(confidence.items(), key=itemgetter(1), reverse=True)

In [19]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print_rule(premise, conclusion, support, confidence, features)

Rule #1
Rule: If a person buys apples they will also buy cheese
 - Confidence: 69.444%
 - Support: 25

Rule #2
Rule: If a person buys cheese they will also buy bananas
 - Confidence: 65.854%
 - Support: 27

Rule #3
Rule: If a person buys bread they will also buy bananas
 - Confidence: 62.963%
 - Support: 17

Rule #4
Rule: If a person buys cheese they will also buy apples
 - Confidence: 60.976%
 - Support: 25

Rule #5
Rule: If a person buys apples they will also buy bananas
 - Confidence: 58.333%
 - Support: 21

